## Modules and data

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
import string
from nltk.corpus import stopwords
from sklearn.decomposition import NMF, TruncatedSVD, PCA
import matplotlib.pyplot as plt 
from sklearn import model_selection
from nltk.stem import PorterStemmer
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import IsolationForest


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_seq_items', 1000)
pd.set_option('display.max_colwidth', -1)
%matplotlib inline

/Users/harry/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [2]:
df = pd.read_csv('../data/external/labelled_headlines_data.csv')

df.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as countries move to brink of war""",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into South Ossetia; footage from fighting (YouTube)',"b'Russian tanks are moving towards the capital of South Ossetia, which has reportedly been completely destroyed by Georgian artillery fire'","b""Afghan children raped with 'impunity,' U.N. official says - this is sick, a three year old was raped and they do nothing""",b'150 Russian tanks have entered South Ossetia whilst Georgia shoots down two Russian jets.',"b""Breaking: Georgia invades South Ossetia, Russia warned it would intervene on SO's side""","b""The 'enemy combatent' trials are nothing but a sham: Salim Haman has been sentenced to 5 1/2 years, but will be kept longer anyway just because they feel like it.""","b'Georgian troops retreat from S. Osettain capital, presumably leaving several hundred people killed. [VIDEO]'",b'Did the U.S. Prep Georgia for War with Russia?',b'Rice Gives Green Light for Israel to Attack Iran: Says U.S. has no veto over Israeli military ops',b'Announcing:Class Action Lawsuit on Behalf of American Public Against the FBI',"b""So---Russia and Georgia are at war and the NYT's top story is opening ceremonies of the Olympics? What a fucking disgrace and yet further proof of the decline of journalism.""","b""China tells Bush to stay out of other countries' affairs""",b'Did World War III start today?',"b'Georgia Invades South Ossetia - if Russia gets involved, will NATO absorb Georgia and unleash a full scale war?'",b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to prevent an Israeli strike on Iran."" Israeli Defense Minister Ehud Barak: ""Israel is prepared for uncompromising victory in the case of military hostilities.""'",b'This is a busy day: The European Union has approved new sanctions against Iran in protest at its nuclear programme.',"b""Georgia will withdraw 1,000 soldiers from Iraq to help fight off Russian forces in Georgia's breakaway region of South Ossetia""",b'Why the Pentagon Thinks Attacking Iran is a Bad Idea - US News &amp; World Report',b'Caucasus in crisis: Georgia invades South Ossetia',"b'Indian shoe manufactory - And again in a series of ""you do not like your work?""'",b'Visitors Suffering from Mental Illnesses Banned from Olympics',"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,"b'Why wont America and Nato help us? If they wont help us now, why did we help them in Iraq?'",b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli training, we're fending off Russia """,b'Georgian army flees in disarray as Russians advance - Gori abandoned to Russia without a shot fired',"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zealand Passports doing in Iraq?',b'Russia angered by Israeli military sale to Georgia',b'An American citizen living in S.Ossetia blames U.S. and Georgian leaders for the genocide of innocent people',b'Welcome To World War IV! Now In High Definition!',"b""Georgia's move, a mistake of monumental proportions """,b'Russia presses deeper into Georgia; U.S. says regime change is goal',b'Abhinav Bindra wins first ever Individual Olympic Gold Medal for India',b' U.S. ship heads for Arctic to define territory',b'Drivers in a Jerusalem taxi station threaten to quit rather than work for their new boss - an Arab',b'The French Team is Stunned by Phelps and the 4x100m Relay Team',b'Israel and the US behind the Georgian aggression?',"b'""Do not believe TV, neither Russian nor Georgian. There are much more victims""'",b'Riots are still going on in Montreal (Canada) because police murdered a boy on Saturday.',b'China to overtake US as largest manufacturer',b'War in South Ossetia [PIC

## Cleaning

### Cleaning function

In [3]:
def clean_news(in_df, num_stories=10, min_df=0.01, max_df=0.99, ngram_range=(1,1), vectorizer=CountVectorizer):

    # set columns
    columns = in_df.loc[:,'Top1':'Top25'].columns.tolist()

    # strip, remove punctuation, make lower case, and remove numbers
    for i in columns:
        in_df[i] = (in_df[i].apply(lambda x: (str(x)).strip())
                      .apply(lambda x: x.lstrip('b'))
                      .apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x.lower()))
                      .apply(lambda x: re.sub('\w*\d\w*', '', x))
                )

    # concatenate the specified number of columns into one column
    temp_dict = {}
    for i in range(in_df.shape[0]):
        temp_string = ''
        for j in columns[0:num_stories]:
            temp_string = temp_string + ' ' + in_df.loc[i, j]

        temp_dict[in_df.loc[i, 'Date']] = [temp_string]

    text_df = pd.DataFrame(temp_dict).T.rename(columns={0: 'text'})

    # instantiate count CountVectorizer
    trans = vectorizer(stop_words='english', min_df=min_df, max_df=max_df, ngram_range=ngram_range)
    X = trans.fit_transform(text_df.text)

    # create final document to feature matrix for modelling
    doc_feat_mat = pd.DataFrame(X.toarray(), columns=trans.get_feature_names(), index = text_df.index)
    doc_feat_mat.index.name = 'date'
    doc_feat_mat = doc_feat_mat.merge(df[['Date','Label']], left_on='date', right_on='Date')
    doc_feat_mat.set_index('Date', inplace=True)
    
    return doc_feat_mat

### Clean data

In [4]:
clean_df = clean_news(df, num_stories=25, min_df=0.05, max_df=0.95, ngram_range=(1,2), vectorizer=TfidfVectorizer)

clean_df.shape

(1989, 991)

## Topic modelling

### Topic modelling function

In [5]:
# function to show words associated with each topic
def topic_model(in_df, n_components = 5, n_results=10, model=NMF):

    feats_df = in_df.drop(columns=['Label'])

    # instantiate model
    the_model = model(n_components = n_components, random_state=10)
    if model == PCA:
        the_model.fit_transform(np.array(feats_df)-np.array(feats_df).mean())
    else:
        the_model.fit_transform(feats_df)

    # make topic to features dataframe
    top_df = pd.DataFrame(the_model.components_.round(4),
                            index = [f'topic_{i}' for i in range(1,(n_components+1))],
                            columns = feats_df.columns.tolist())

    # print results
    for i in range(1,(n_components+1)):
        results = top_df.loc[f'topic_{i}',:].sort_values(ascending=False).index.tolist()[0:n_results]
        print(f'topic_{i}:', results)
        print(' ')

### Model topics

In [6]:
topic_model(clean_df, n_components = 15, n_results=20, model=NMF)

topic_1: ['police', 'war', 'afghanistan', 'pakistan', 'people', 'man', 'uk', 'death', 'iraq', 'years', 'afghan', 'children', 'taliban', 'video', 'killed', 'british', 'soldiers', 'dead', 'world', 'israeli']
 
topic_2: ['gaza', 'israel', 'israeli', 'hamas', 'palestinians', 'israels', 'aid', 'war', 'palestinian', 'civilians', 'crimes', 'war crimes', 'israelis', 'ship', 'jewish', 'strip', 'children', 'soldiers', 'arab', 'white']
 
topic_3: ['ukraine', 'russia', 'russian', 'ukrainian', 'putin', 'nato', 'military', 'says', 'troops', 'russias', 'sanctions', 'eastern', 'forces', 'vladimir', 'poland', 'president', 'moscow', 'crisis', 'vladimir putin', 'war']
 
topic_4: ['korea', 'north', 'north korea', 'south', 'korean', 'south korea', 'north korean', 'kim', 'nuclear', 'koreas', 'missile', 'china', 'test', 'missiles', 'launch', 'new', 'says', 'border', 'military', 'news']
 
topic_5: ['snowden', 'nsa', 'edward', 'edward snowden', 'spying', 'surveillance', 'asylum', 'data', 'says', 'spy', 'docume

In [7]:
topic_names = ['human_rights', 
               'israel_palestine', 
               'north_south_korea',
               'ukraine_russia', 
               'snowden_nsa', 
               'wikileaks_assange', 
               'islam_terrorism', 
               'saudi_oil',
               'nuclear_disaster',
               'egypt_protests', 
               'china_eu',
               'climate_change',
               'world_leaders',
               'war_afghan_iraq',  
               'child_abuse']

### Topic to document function

In [8]:
# function to give document to topics relationships
def topic_document(in_df, topic_names, n_components = 5, model=NMF):

    feats_df = in_df.drop(columns=['Label'])

    # instantiate model
    the_model = model(n_components = n_components, random_state=10)
    if model == PCA:
        values = the_model.fit_transform(np.array(feats_df)-np.array(feats_df).mean())
    else:
        values = the_model.fit_transform(feats_df)

    top_df = pd.DataFrame(values, index = feats_df.index, columns = topic_names)
    top_df = top_df.merge(clean_df.reset_index()[['Date','Label']], left_on='Date', right_on='Date')
    top_df.set_index('Date', inplace=True)
    top_df = top_df[ ['Label'] + [ col for col in top_df.columns if col != 'Label' ] ]
    
    # make document to topics dataframe
    return top_df

### Documents to topics

In [9]:
topics_df = topic_document(clean_df, topic_names, n_components = 15, model=NMF)

topics_df.head()

,Label,human_rights,israel_palestine,north_south_korea,ukraine_russia,snowden_nsa,wikileaks_assange,islam_terrorism,saudi_oil,nuclear_disaster,egypt_protests,china_eu,climate_change,world_leaders,war_afghan_iraq,child_abuse
Date,,,,,,,,,,,,,,,,
2008-08-08,0,0.053818,0.027375,0.088490,0.032158,0.0,0.000000,0.000000,0.0,0.0,0.005338,0.000000,0.098862,0.000000,0.0,0.000000
2008-08-11,1,0.047810,0.047101,0.081530,0.001167,0.0,0.000000,0.006818,0.0,0.0,0.002345,0.000000,0.038489,0.000000,0.0,0.022559
2008-08-12,0,0.061799,0.010213,0.083280,0.024282,0.0,0.000553,0.000000,0.0,0.0,0.007881,0.000000,0.027873,0.000000,0.0,0.020119
2008-08-13,0,0.002087,0.032581,0.087196,0.005366,0.0,0.000000,0.000000,0.0,0.0,0.069832,0.000000,0.116780,0.019792,0.0,0.000000
2008-08-14,1,0.051451,0.002702,0.080644,0.024874,0.0,0.000000,0.000000,0.0,0.0,0.024536,0.091382,0.006592,0.000000,0.0,0.000000


### Sanity check

In [10]:
topics_df.north_south_korea.sort_values(ascending=False).head(5)

Date
2014-03-03    0.292322
2014-03-04    0.280621
2014-05-02    0.271492
2014-04-17    0.260840
2014-04-15    0.254473
Name: north_south_korea, dtype: float64

In [11]:
df[df.Date == '2013-04-10'].squeeze()

Date     2013-04-10                                                                                                                                                                                                                                          
Label    1                                                                                                                                                                                                                                                   
Top1     nasabacked fusion engine could cut mars trip down to  days                                                                                                                                                                                          
Top2     germany gives japan its backing as n korea tensions rise                                                                                                                                                                             

## Sentiment analysis

### Sentiment data

In [12]:
sentiment_df = pd.read_csv('../data/processed/sentiment_data.csv', index_col = 'Date').loc[:,'Subjectivity':'Negative']

sentiment_df.head()

,Subjectivity,Objectivity,Positive,Neutral,Negative
Date,,,,,
2008-08-08,75.000000,25.000000,18.750000,25.000000,56.250000
2008-08-11,83.333333,16.666667,41.666667,16.666667,41.666667
2008-08-12,56.250000,43.750000,18.750000,43.750000,37.500000
2008-08-13,38.461538,61.538462,15.384615,61.538462,23.076923
2008-08-14,45.454545,54.545455,36.363636,54.545455,9.090909


### Combine sentiment data with clean word data

In [13]:
combo_sent_df = clean_df.merge(sentiment_df, left_on=clean_df.index, right_on=sentiment_df.index)
combo_sent_df.set_index('key_0', inplace=True)
combo_sent_df.index.name = 'Date'
combo_sent_df = combo_sent_df[ ['Label'] + [ col for col in combo_sent_df.columns if col != 'Label' ] ]

combo_sent_df.head()

,Label,able,abuse,access,according,accused,accuses,act,action,activist,activists,admits,afghan,afghanistan,africa,african,age,agency,ago,agreement,ahead,aid,air,aircraft,airport,al,alive,allegations,alleged,allegedly,allow,allowed,amazon,ambassador,america,american,americans,amid,amp,ancient,animals,announced,announces,anonymous,approved,arab,arabia,area,armed,arms,army,arrest,arrested,arrests,asia,asks,assad,assange,assault,asylum,attack,attacked,attacks,attempt,australia,australian,authorities,avoid,away,baby,bad,ban,bank,banks,banned,bans,base,battle,bay,bbc,bbc news,beaten,beijing,believe,best,better,bid,big,biggest,billion,billions,black,block,blood,bodies,body,bomb,bombing,bombs,border,borders,boy,brazil,brazilian,break,bring,britain,britains,british,build,building,built,bus,bush,business,called,calling,calls,cameron,camp,campaign,canada,canadas,canadian,cancer,cannabis,capital,captured,car,carrying,cars,case,cases,cash,catholic,caught,cause,caused,central,change,charge,charged,charges,chemical,chief,child,children,china,chinas,chinese,christian,christians,church,cia,cities,citizens,city,civil,civilians,claim,claims,clean,climate,climate change,close,coal,coast,collapse,come,comes,coming,community,companies,company,confirmed,confirms,conflict,construction,continue,control,controversial,convicted,corruption,cost,council,countries,country,countrys,court,crackdown,crash,crime,crimes,criminal,crisis,cuba,cup,currency,cut,cuts,damage,dangerous,data,daughter,david,david cameron,day,days,dead,deadly,deal,death,deaths,debate,debt,decades,decision,declared,declares,defence,defense,demand,demands,democracy,denies,despite,destroy,destroyed,detained,did,didnt,die,died,dies,disaster,discovered,doctors,documents,does,doesnt,doing,dollars,dont,dozens,drone,drug,drugs,dubai,dutch,earth,...,russias,said,sanctions,satellite,saudi,saudi arabia,save,say,saying,says,scandal,school,schools,scientists,scotland,sea,search,second,secret,security,security forces,seek,seeks,seen,seized,sell,selling,send,sending,senior,sent,sentence,sentenced,service,services,set,settlement,seven,sex,sexual,ship,shooting,shot,shot dead,shows,shut,sign,signs,single,site,sites,small,snowden,social,society,solar,sold,soldier,soldiers,son,soon,source,south,south africa,south korea,southern,space,spain,spanish,special,species,speech,spread,spy,spying,square,staff,stand,start,started,state,states,station,stay,step,stolen,stop,story,street,streets,strike,strikes,strip,student,students,study,sudan,suicide,sunday,supply,support,supreme,supreme court,surveillance,suspected,sweden,swedish,swiss,syria,syrian,taken,takes,taking,taliban,talks,target,targets,tax,team,tells,tens,territory,terror,terrorism,terrorist,terrorists,test,thats,theyre,think,thought,thousands,threat,threatened,threatens,threats,thursday,ties,time,times,today,told,toll,tony,took,torture,tortured,total,town,trade,train,treatment,trial,tried,troops,try,trying,tuesday,turkey,turkish,turn,turned,turns,tv,twitter,uk,ukraine,ukrainian,union,united,united nations,united states,university,urges,use,used,users,using,vatican,venezuela,victim,victims,video,village,violence,violent,virus,visit,vladimir,vladimir putin,vote,votes,wall,want,wanted,wants,war,war crimes,warned,warning,warns,washington,watch,water,waters,way,weapons,web,website,websites,wednesday,week,weeks,west,west bank,western,white,wife,wikileaks,win,wins,woman,women,wont,work,workers,working,world,world war,worlds,worlds largest,worldwide,worse,worst,worth,wounded,wrong,year,year old,years,years ago,yemen,young,zealand,zone,Subjectivity,Objectivity,Positive,Neutral,Negative
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

In [14]:
combo_sent_df.dropna(inplace=True)

combo_sent_df.shape

(1986, 996)

### Make topic to sentiment dataframe

In [15]:
# fit model on word columns transposed 
word_cols = np.array(combo_sent_df.loc[:, 'abuse':'zealand']).T

word_cols.shape

(988, 1986)

In [16]:
sent_model = NMF(n_components = 15)
sent_model.fit(word_cols)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=15, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [17]:
sent_cols = np.array(combo_sent_df.loc[:, 'Subjectivity':'Negative']).T

sent_cols.shape

(5, 1986)

In [18]:
# use same model to transform sentiment columns
sent_model.transform(sent_cols).shape

(5, 15)

In [19]:
# set new topic titles for presentation

topic_titles = ['Human rights', 
               'Israel/Palestine', 
               'North Korea/South Korea',
               'Russia/Ukraine', 
               'NSA/Edward Snowden', 
               'Wikileaks/Julian Assange', 
               'Islam/Terrorism', 
               'Oil/Saudi Arabia',
               'Nuclear weapons',
               'Egypt/Protest', 
               'Asian countries',
               'Climate change',
               'World leaders',
               'War/Afghanistan/Iraq',  
               'Child abuse']

In [20]:
sent_topic_df = pd.DataFrame(sent_model.transform(sent_cols).round(4).T, 
                             columns = sentiment_df.columns.tolist(),
                             index = topic_titles)

sent_topic_df.head(15)

,Subjectivity,Objectivity,Positive,Neutral,Negative
Human rights,200.1090,313.0574,74.4099,313.0574,125.6991
Israel/Palestine,81.4265,157.1899,35.1617,157.1899,46.2648
North Korea/South Korea,122.9068,51.9941,49.5040,51.9941,73.4029
Russia/Ukraine,134.8396,76.7891,52.6987,76.7891,82.1408
NSA/Edward Snowden,104.2762,63.7732,52.4142,63.7732,51.8620
Wikileaks/Julian Assange,74.2477,67.2129,44.1200,67.2129,30.1276
Islam/Terrorism,205.1366,17.8231,52.5925,17.8231,152.5440
Oil/Saudi Arabia,156.7698,73.4212,46.1389,73.4212,110.6305
Nuclear weapons,299.7979,112.5052,100.9549,112.5052,198.8427
Egypt/Protest,126.5668,132.1592,44.4171,132.1592,82.1495


In [21]:
# make new columns for vizualization
sent_topic_df['Net_Positive'] = sent_topic_df.apply(lambda row: row['Positive'] - row['Negative'], axis=1)
sent_topic_df['Net_Objectivity'] = sent_topic_df.apply(lambda row: row['Objectivity'] - row['Subjectivity'], axis=1)
sent_topic_df['Polarity'] = sent_topic_df.apply(lambda row: (row['Positive'] + row['Negative'])/2 - row['Neutral'], axis=1)

In [22]:
sent_topic_df.mean()

Subjectivity       147.160453
Objectivity        105.136347
Positive           52.542207 
Neutral            105.136347
Negative           94.618220 
Net_Positive      -42.076013 
Net_Objectivity   -42.024107 
Polarity          -31.556133 
dtype: float64

In [23]:
sent_topic_df.to_csv('sent_topic_data.csv')

## Anomaly detection

### Find anomalies with isolation forest

In [24]:
topics_df.head()

,Label,human_rights,israel_palestine,north_south_korea,ukraine_russia,snowden_nsa,wikileaks_assange,islam_terrorism,saudi_oil,nuclear_disaster,egypt_protests,china_eu,climate_change,world_leaders,war_afghan_iraq,child_abuse
Date,,,,,,,,,,,,,,,,
2008-08-08,0,0.053818,0.027375,0.088490,0.032158,0.0,0.000000,0.000000,0.0,0.0,0.005338,0.000000,0.098862,0.000000,0.0,0.000000
2008-08-11,1,0.047810,0.047101,0.081530,0.001167,0.0,0.000000,0.006818,0.0,0.0,0.002345,0.000000,0.038489,0.000000,0.0,0.022559
2008-08-12,0,0.061799,0.010213,0.083280,0.024282,0.0,0.000553,0.000000,0.0,0.0,0.007881,0.000000,0.027873,0.000000,0.0,0.020119
2008-08-13,0,0.002087,0.032581,0.087196,0.005366,0.0,0.000000,0.000000,0.0,0.0,0.069832,0.000000,0.116780,0.019792,0.0,0.000000
2008-08-14,1,0.051451,0.002702,0.080644,0.024874,0.0,0.000000,0.000000,0.0,0.0,0.024536,0.091382,0.006592,0.000000,0.0,0.000000


In [25]:
X = np.array(topics_df)
iso = IsolationForest(random_state=10).fit(X)

In [26]:
dates = topics_df.index.tolist()
anomalies = iso.predict(X)

dates_values = {}
for i, date in enumerate(dates):
    value = anomalies[i]
    
    dates_values[date] = [value]
    
dates_values

{'2008-08-08': [1],
 '2008-08-11': [1],
 '2008-08-12': [1],
 '2008-08-13': [1],
 '2008-08-14': [1],
 '2008-08-15': [1],
 '2008-08-18': [1],
 '2008-08-19': [1],
 '2008-08-20': [1],
 '2008-08-21': [1],
 '2008-08-22': [1],
 '2008-08-25': [1],
 '2008-08-26': [1],
 '2008-08-27': [1],
 '2008-08-28': [1],
 '2008-08-29': [1],
 '2008-09-02': [1],
 '2008-09-03': [1],
 '2008-09-04': [1],
 '2008-09-05': [1],
 '2008-09-08': [1],
 '2008-09-09': [1],
 '2008-09-10': [1],
 '2008-09-11': [1],
 '2008-09-12': [1],
 '2008-09-15': [1],
 '2008-09-16': [1],
 '2008-09-17': [1],
 '2008-09-18': [1],
 '2008-09-19': [1],
 '2008-09-22': [1],
 '2008-09-23': [1],
 '2008-09-24': [1],
 '2008-09-25': [1],
 '2008-09-26': [1],
 '2008-09-29': [1],
 '2008-09-30': [1],
 '2008-10-01': [1],
 '2008-10-02': [1],
 '2008-10-03': [1],
 '2008-10-06': [1],
 '2008-10-07': [1],
 '2008-10-08': [1],
 '2008-10-09': [1],
 '2008-10-10': [1],
 '2008-10-13': [1],
 '2008-10-14': [1],
 '2008-10-15': [1],
 '2008-10-16': [1],
 '2008-10-17': [1],


In [27]:
values_df = pd.DataFrame(dates_values).T.rename(columns={0: "value"})
anomalies = values_df[values_df['value'] == -1].index.tolist()

anomalies

['2011-03-10',
 '2013-06-10',
 '2013-06-13',
 '2013-06-14',
 '2013-06-24',
 '2013-06-27',
 '2013-07-01',
 '2013-09-16',
 '2013-11-11',
 '2013-12-26',
 '2014-07-15',
 '2014-07-17',
 '2014-09-04',
 '2014-10-13',
 '2015-01-23',
 '2015-03-10',
 '2016-03-08',
 '2016-04-14',
 '2016-04-20']

### Anomalies data

In [28]:
anomalies_df = df[df['Date'].isin(anomalies)]

anomalies_df.head(100)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
651,2011-03-10,0,dalai lama to retire citing that the tibetan people deserve a freely elected leader,julian assange police investigator a friend of sex assault accuser,france recognises rebels as libyan government,interrogator in assange case is friends with one of the two women who reported wikileaksfounder of sexual abuse,saudi police open fire at protest,libya sky news march incredible footage the battle for zawiyah libya video,police fire on protesters in saudi arabia,france becomes first country to recognize libyan rebels,saudi arabian police open fire at rally in the eastern part of the kingdom which means its likely lead by shiites and is also directly on top of the vast oil reserves the worlds largest,interrogator in the assange case friend with woman accusing wikileaks founder,an abandoned and unfinished skyscraper in caracas venezuela houses an organized community of roughly squatters,dalai lama steps down as tibetan leader,cameroon bans twitter in an attempt to stave off opposition protests,north korea nears completion of emp bomb could permanently damage electronics if exploded miles above ground fixed link,nerve gas used on yemen protesters,palestinian family ordered to share east jerusalem home with israelis,protests and violence spread to saudi arabia police open fire on hundreds of protesters with guns and stun grenades in eastern city of qatif,reports saudi police fire at protesters,russia announces ban on arms sales to libya,guatemala to sue us for intentionally infecting guatemalans with syphilis in the,christians in egypt attacked dead,libya nicolas sarkozy to urge targeted air strikes,yemeni army storms university wounding,royal bank of scotland chief executive got pay package even though the bailedout bank lost more than,britain demands gaddafi ultimatum after bbc staff tortured in libya
1216,2013-06-10,0,edward snowden the whistleblower behind revelations of nsa surveillance,icelandic legislator im ready to help nsa whistleblower edward snowden seek asylum,germany most snooped eu country by us,why the nsas secret online surveillance should scare you george orwell isnt who you should be reading to understand the dangers inherent to the nsas dragnet youd be better off turning to famous french social theorist michel foucault,why edward snowdens flight to hong kong might be brilliant the nsa whistleblower could exploit a loophole in the chinese territorys asylum system to buy himself some valuable time,revelations of a huge secret us internet spying program have raised awkward questions for allies forced to explain whether they let washington spy on their citizens or benefited from snooping that would be illegal at home,why canadians should be demanding answers about secret surveillance programs canada has similar disclosure provisions as those found in the usa patriot act,after prism boundless informant tool comes to light meet the us nsas global intelligence tracking tool boundless informant the latest intelligence secret exposed by leaked information,turkey erupts the new young turks many are strikingly young but there are plenty of older turks many secularminded some overtly pious there are gays armenians anarchists and atheists,technology giants struggle to maintain credibility over nsa surveillance scandal,syrian rebels execute teenage boy for heresy an alqaedaaffiliated opposition group has allegedly executed a teenage boy in syria in front of his family the syrian observatory for human rights reports the boy was shot by the group for supposedly blaspheming,a senior figure in hong kong law enforcement has suggested nsa whistleblower edward snowden should leave the city,microsoft admits patriot act can access eubased cloud data,syrian rebels execute teenager in aleppo for blasphemy,canadian marijuana seed magnate and political dissident marc emery in s